In [1]:
from focus_stack import Job, ActionList, file_folder
from termcolor import colored, cprint
import focus_stack as stk
import os
import time

In [2]:
class PathJob(Job):
    def __init__(self, name, wdir):
        assert  os.path.exists(wdir), 'Path does not exist: ' + wdir
        self.wdir = wdir
        Job.__init__(self, name)

In [3]:
class FramesRefActions(ActionList):
    def __init__(self, name, wdir, input_path, output_path='', ref_idx=-1, step_align=True):
        assert  os.path.exists(wdir), 'Path does not exist: ' + wdir
        self.wdir = wdir
        if output_path=='': output_path = name
        self.input_dir = wdir + input_path
        self.output_dir = wdir + output_path
        if not os.path.exists( self.output_dir): os.makedirs( self.output_dir)
        self.fnames = file_folder(self.input_dir)
        ActionList.__init__(self, name, len(self.fnames))
        self.ref_idx = ref_idx
        if ref_idx == -1: self.ref_idx = len(self.fnames) // 2
        self.step_align = step_align
    def run_step(self):
        cprint("action: {} ".format(self.fnames[self.count - 1]), "blue", end='\r')
        if self.count == 1:
            self.__idx = self.ref_idx if self.step_align else 0
            self.__ref_idx = self.ref_idx
            self.__idx_step = +1
        self.run_frame(self.__idx, self.__ref_idx)
        ll = len(self.fnames)
        if(self.__idx < ll):
            if self.step_align: self.__ref_idx = self.__idx
            self.__idx += self.__idx_step
        if(self.__idx == ll):
            self.__idx = self.ref_idx - 1
            if self.step_align: self.__ref_idx = self.ref_idx
            self.__idx_step = -1

In [11]:
class AlignLayers(FramesRefActions):
    ALIGN_HOMOGRAPHY = "homography"
    ALIGN_RIGID = "rigid"
    def __init__(self, wdir, name, input_path, output_path='', ref_idx=-1, step_align=True, detector_method='SIFT', descriptor_method='SIFT', match_method='KNN', flann_idx_kdtree=0, match_threshold=0.7, method=ALIGN_HOMOGRAPHY, plot=False):
        FramesRefActions.__init__(self, name, wdir, input_path, output_path, ref_idx, step_align)
        self.detector_method = detector_method
        self.descriptor_method = descriptor_method
        self.match_method = match_method
        self.flann_idx_kdtree = flann_idx_kdtree
        self.match_threshold = match_threshold
        self.method = method
        self.plot = plot
    def run_frame(self, idx, ref_idx):
        print("frame: {}, index: {}, reference: {}, align file: {}                    ".format(self.count, idx, ref_idx, self.fnames[idx]), end='\r')
        ref_dir = self.output_dir if self.step_align else self.input_dir
        stk.img_align(self.fnames[ref_idx], self.fnames[idx], ref_dir, self.input_dir, self.output_dir, self.detector_method, self.descriptor_method, self.match_method, self.flann_idx_kdtree, self.match_threshold, self.method, self.plot, verbose=False)

In [12]:
class BalanceLayers(FramesRefActions):
    BALANCE_LUMI = "lumi"
    BALANCE_RGB = "rgb"
    BALANCE_SV = "sv"
    BALANCE_LS = "ls"
    def __init__(self, wdir, name, input_path, output_path='', ref_idx=-1, method=BALANCE_LUMI, mask_radius=-1, i_min=0, i_max=255, plot=False):
        FramesRefActions.__init__(self, name, wdir, input_path, output_path, ref_idx, step_align=False)
        self.method = method
        self.mask_radius = mask_radius
        self.i_min = i_min
        self.i_max = i_max
        self.plot = plot
    def run_frame(self, idx, ref_idx):
        print("frame: {}, index: {}, reference: {}, balance file: {}                    ".format(self.count, idx, ref_idx, self.fnames[idx]), end='\r')
        if self.method ==BalanceLayers.BALANCE_LUMI:
            stk.img_lumi_balance(self.fnames[ref_idx], self.fnames[idx], self.input_dir, self.output_dir, self.mask_radius, self.i_min, self.i_max, self.plot, verbose=False)
        elif self.method == BalanceLayers.BALANCE_RGB:
            stk.img_lumi_balance_rgb(self.fnames[ref_idx], self.fnames[idx], self.input_dir, self.output_dir, self.mask_radius, self.i_min, self.i_max, self.plot, verbose=False)
        elif self.method == BalanceLayers.BALANCE_SV:
            stk.img_lumi_balance_hsv(self.fnames[ref_idx], self.fnames[idx], self.input_dir, self.output_dir, self.mask_radius, self.i_min, self.i_max, self.plot, verbose=False)
        else: 
            raise Exceltion("invalid method: " + self.method)

In [7]:
job = PathJob("job", "E:/Focus stacking/2024-03-17 - Baltic Diptera/A/")
align = AlignLayers(job.wdir, "align", "Immagini modificate", method=AlignLayers.ALIGN_RIGID)
balance = BalanceLayers(job.wdir, "balance", "align", method=BalanceLayers.BALANCE_LUMI, mask_radius=0.8, i_min=10, i_max=255)

In [9]:
job.add_action(align)
job.add_action(balance)

In [10]:
job.run()

run align


align:   0%|          | 0/76 [00:00<?, ?it/s]

align: elapsed time: 00:02:19.71s                          
run balance


balance:   0%|          | 0/76 [00:00<?, ?it/s]

balance: elapsed time: 00:00:28.98s                          
job: elapsed time: 00:02:48.68s                    
